# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [3]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [4]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [5]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [6]:
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [7]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [9]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [10]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 6001, 79458, 33759]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [11]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [12]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [13]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

prod_dist = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot, 'euclidean'))),
                         index=prod_cust_pivot.index, columns=prod_cust_pivot.index)

prod_dist.head(3)

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.881127,0.882219,0.884604,0.890449,0.879859,0.885648,0.880852,0.886501,0.890573,...,0.884117,0.888124,0.886567,0.879578,0.875696,0.878427,0.886163,0.886564,0.887956,0.878223
"Appetizer - Mini Egg Roll, Shrimp",0.881127,1.000000,0.883877,0.880604,0.887930,0.881577,0.885241,0.890196,0.888646,0.883484,...,0.887569,0.887831,0.888033,0.881019,0.878720,0.871915,0.888702,0.890837,0.887661,0.882616
Appetizer - Mushroom Tart,0.882219,0.883877,1.000000,0.884708,0.883266,0.884124,0.886495,0.886072,0.887707,0.881019,...,0.886796,0.885968,0.885056,0.887445,0.882789,0.879286,0.890062,0.889960,0.889397,0.881077


### Step 2: Get the products purchased for a specific customer of your choice.

In [14]:
#customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
spec_customer = customer_products[customer_products['CustomerID'] ==264]
spec_customer.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [19]:
list_prod_spec_customer = list(spec_customer['ProductName'])
ranked_products_list=[]
for prod in list_prod_spec_customer:
    similar_prod = list(prod_dist[prod].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['ProductName'].isin(similar_prod)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    ranked_products_list.append(ranked_products['ProductName'])

ranked_products_flat = [y for x in ranked_products_list for y in x]

# A FAIRE : ranked_products_flat=[rank for prod in list_prod_spec_customer for rank in ranked_products_flat if rank != prod]
ranked_products_flat = [item for item in ranked_products_flat if item not in list(list_prod_spec_customer)]
ranked_products_flat

['Oil - Shortening - All - Purpose',
 'Scallops - Live In Shell',
 'Juice - Orange',
 'Cheese - Mozzarella',
 'Meldea Green Tea Liquor',
 'Spinach - Baby',
 'Bread - Rye',
 'Veal - Osso Bucco',
 'Beans - Wax',
 'Broom - Corn',
 'Cod - Black Whole Fillet',
 'Broom - Corn',
 'Rosemary - Primerba, Paste',
 'Sun - Dried Tomatoes',
 'Yogurt - Blueberry, 175 Gr',
 'Oil - Shortening - All - Purpose',
 'French Pastry - Mini Chocolate',
 'Scallops - Live In Shell',
 'Mussels - Cultivated',
 'Salmon - Atlantic, Skin On',
 'Rosemary - Primerba, Paste',
 'Sun - Dried Tomatoes',
 'Vinegar - Tarragon',
 'Pernod',
 'Chocolate - Compound Coating',
 'Rosemary - Primerba, Paste',
 'Sunflower Seed Raw',
 'Yoghurt Tubes',
 'Cookies - Assorted',
 'Bread - Italian Roll With Herbs',
 'Beans - Wax',
 'Initation Crab Meat',
 'Barramundi',
 'Beef - Montreal Smoked Brisket',
 'Veal - Osso Bucco',
 'Coffee - Irish Cream',
 'Artichokes - Jerusalem',
 'Appetizer - Mushroom Tart',
 'Soup Knorr Chili With Beans',
 'V

### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

In [20]:
top_five = pd.Series(ranked_products_flat).value_counts().head()
top_five

Rosemary - Primerba, Paste          14
Sun - Dried Tomatoes                11
Spinach - Baby                      11
Oil - Shortening - All - Purpose    10
V8 - Berry Blend                    10
dtype: int64

### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [32]:
reco_for_customer={}
def recommandation(customer): 
    spec_customer = customer_products[customer_products['CustomerID'] ==customer]
    list_prod_spec_customer = list(spec_customer['ProductName'])
    ranked_products_list=[]
    for prod in list_prod_spec_customer:
        similar_prod = list(prod_dist[prod].sort_values(ascending=False)[1:].head().index)
        sim_cust_prod = customer_products[customer_products['ProductName'].isin(similar_prod)]
        grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
        ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
        ranked_products_list.append(ranked_products['ProductName'])
    
    ranked_products_flat = [y for x in ranked_products_list for y in x]
    top_five = pd.Series(ranked_products_flat).value_counts().head()
    top_five_dict = top_five.to_dict().keys()
    #new_reco_dico=dict((i+1,top_five_dict) for i in enumerate(sorted(top_five_dict)))
    reco_for_customer.update({customer:top_five_dict})
    #print(reco_for_customer)
    
        
CustomerID_list=set(data.CustomerID)

for customer in CustomerID_list:
    recommandation(customer)

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [33]:
prod_recs = pd.DataFrame.from_dict(reco_for_customer, orient='index').reset_index()
prod_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
prod_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,83973,"Rosemary - Primerba, Paste",Oil - Shortening - All - Purpose,Rice - Jasmine Sented,Cookies - Assorted,Sun - Dried Tomatoes
1,59399,"Rosemary - Primerba, Paste",Sun - Dried Tomatoes,Oil - Shortening - All - Purpose,Spinach - Baby,Cookies - Assorted
2,92168,Hersey Shakes,Cookies - Assorted,Spinach - Baby,Sun - Dried Tomatoes,Flavouring - Orange
3,49159,Oil - Shortening - All - Purpose,"Rosemary - Primerba, Paste",Spinach - Baby,Towels - Paper / Kraft,Sun - Dried Tomatoes
4,18441,V8 - Berry Blend,"Rosemary - Primerba, Paste",Oil - Shortening - All - Purpose,Cookies - Assorted,Sun - Dried Tomatoes


## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [59]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

In [60]:
new_customer = pd.DataFrame.from_dict(new_customer, orient='index').reset_index()
new_customer.columns = ['ProductName', 'Quantity']
new_customer['CustomerID']=customer_products['CustomerID'].max()+1
new_customer = new_customer[['CustomerID','ProductName', 'Quantity']]
customer_products = customer_products.append(new_customer,ignore_index=True)
customer_products.tail()

,CustomerID,ProductName,Quantity
63635,98201,French Pastry - Mini Chocolate,4
63636,98201,Snapple Lemon Tea,5
63637,98201,"Pepper - White, Ground",2
63638,98201,Spinach - Baby,5
63639,98201,"Sole - Dover, Whole, Fresh",4


In [61]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)
cust_prod_pivot = prod_cust_pivot.T
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.tail()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,98201
CustomerID,,,,,,,,,,,,,,,,,,,,,
98069,0.852848,0.868990,0.859773,0.851219,0.855052,0.852574,0.857118,0.851869,0.855343,0.855701,...,0.855797,0.855409,0.847469,0.861720,0.862919,1.000000,0.859010,0.864271,0.858525,0.754039
98159,0.847637,0.857312,0.854246,0.852260,0.856608,0.852782,0.854378,0.851396,0.855557,0.851331,...,0.858006,0.853659,0.845236,0.855405,0.855458,0.859010,1.000000,0.861266,0.854102,0.753963
98185,0.856465,0.861508,0.864699,0.860721,0.865269,0.858103,0.859154,0.860937,0.860604,0.853331,...,0.852713,0.860604,0.854012,0.859133,0.858954,0.864271,0.861266,1.000000,0.864371,0.751232
98200,0.851614,0.857184,0.853668,0.854377,0.862253,0.859522,0.855788,0.852546,0.851431,0.856960,...,0.852667,0.858412,0.848306,0.856712,0.851527,0.858525,0.854102,0.864371,1.000000,0.752741
98201,0.751984,0.750250,0.751996,0.745769,0.762370,0.748981,0.747362,0.748355,0.747617,0.748731,...,0.750882,0.751649,0.744316,0.753567,0.747676,0.754039,0.753963,0.751232,0.752741,1.000000


In [62]:
recommendations = {}

similar_cust = list(cust_dist[98201].sort_values(ascending=False)[1:].head().index)
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()

merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[98201]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recommendations[98201] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,98201,Ezy Change Mophandle,Foam Cup 6 Oz,"Coconut - Shredded, Sweet",Beets - Mini Golden,Veal - Sweetbread


### Step 8: Recommend 5 products to this new customer using a item similarity approach.

In [63]:
prod_dist = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot, 'euclidean'))),
                         index=prod_cust_pivot.index, columns=prod_cust_pivot.index)

In [68]:
reco_for_customer={}
def recommandation(customer): 
    spec_customer = customer_products[customer_products['CustomerID'] ==customer]
    list_prod_spec_customer = list(spec_customer['ProductName'])
    ranked_products_list=[]
    for prod in list_prod_spec_customer:
        similar_prod = list(prod_dist[prod].sort_values(ascending=False)[1:].head().index)
        sim_cust_prod = customer_products[customer_products['ProductName'].isin(similar_prod)]
        grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
        ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
        ranked_products_list.append(ranked_products['ProductName'])
    
    ranked_products_flat = [y for x in ranked_products_list for y in x]
    top_five = pd.Series(ranked_products_flat).value_counts().head()
    top_five_dict = top_five.to_dict().keys()
    #new_reco_dico=dict((i+1,top_five_dict) for i in enumerate(sorted(top_five_dict)))
    reco_for_customer.update({customer:top_five_dict})
    #print(spec_customer)

    
recommandation(98201)
reco_for_customer

{98201: dict_keys(['Rice - Jasmine Sented', 'Beef - Chuck, Boneless', 'Towels - Paper / Kraft', 'Cookies - Assorted', 'Spinach - Baby'])}

In [69]:
prod_recs = pd.DataFrame.from_dict(reco_for_customer, orient='index').reset_index()
prod_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
prod_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,98201,Rice - Jasmine Sented,"Beef - Chuck, Boneless",Towels - Paper / Kraft,Cookies - Assorted,Spinach - Baby
